template from this: https://medium.com/@prateekgaurav/step-by-step-content-based-recommendation-system-823bbfd0541c

In [1]:
#Import all the required packages
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

/var/folders/90/6fny8sws63z9h6ktyqf5vr6r0000gn/T/ipykernel_24741/3398727433.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Load the bills.csv file into a Pandas dataframe
bills = pd.read_csv('/Users/qing/desktop/projects/LegisLink/billdata2.csv')

In [3]:
privatebilllist = bills.copy()
privatebilllist['Title'] = bills['Title'].str.lower() #for easier searching, lowercase version

~~~
Bag of Words (BoW) is a way of representing text data in numerical form, where
each word in a text document is treated as a separate feature. In the context
of movie recommendation, BoW is typically used to represent movie descriptions,
summaries, or tag information.

When creating a BoW representation for bills in the MovieLens dataset, you
would process the text data for each movie (e.g., movie tag, title, or
summary) and create a vocabulary of all the unique words. You would then use
this vocabulary to represent each movie as a numerical vector, with the length
of the vector being equal to the size of the vocabulary.

Each element in the vector would correspond to a word in the vocabulary, and
the value of each element would indicate the frequency of that word in the
movie description, summary, or tag information.

A simple example of a BoW representation for two bills might look like this:

Movie 1: "Action", "Adventure", "Thriller"
Movie 2: "Comedy", "Romance", "Drama"

Vocabulary: "Action", "Adventure", "Thriller", "Comedy", "Romance", "Drama"

Movie 1 vector: [1, 1, 1, 0, 0, 0]
Movie 2 vector: [0, 0, 0, 1, 1, 1]

In this example, the vocabulary includes all the unique words from the movie
tags, and the vectors for each movie represent the presence or absence of
each word in the vocabulary for that movie.
~~~

In [4]:
# Extract the movie titles and tags into separate lists
titles = bills['Title'].tolist()
tags = bills['Tags'].str.split("|").tolist()

In [5]:
# Create a bag of words representation of the movie tags
def create_bow(tag_list):
    bow = {}
    for tag in tag_list:
        bow[tag] = 1
    return bow

In [6]:
# Create a list of bags of words representations of the movie tags
bags_of_words = [create_bow(movie_tags) for movie_tags in tags]

In [7]:
# Create a dataframe to store the bags of words representation of the movie tags
tag_df = pd.DataFrame(bags_of_words, index=titles).fillna(0)

In [8]:
#tag_df

In [9]:
# Calculate the cosine similarity matrix between the bills
cosine_similarity = cosine_similarity(tag_df)

# Create a dataframe with the cosine similarity scores
similarity_df = pd.DataFrame(cosine_similarity, index=tag_df.index, columns=tag_df.index)

In [10]:
#similarity_df

In [11]:
# usersimilarities = pd.DataFrame(0, index=[0], columns=similarity_df.columns) #initialize preferences to all 0

# x = 'tax'
# #pd.concat([bills.loc[bills['Tags'].str.contains(x, case=False)], bills.loc[bills['Title'].str.contains(x, case=False)]]) #filter bills by tag
# similartags = privatebilllist.loc[privatebilllist['Tags'].str.contains(x.lower(), case=False)].index.tolist()
# for x in privatebilllist.loc[privatebilllist['Title'].str.contains(x.lower(), case=False)].index.tolist():
#   if x not in similartags:
#     similartags.append(x) #add any keywords in title also

# similartags
# for y in range(len(similartags)):
#    usersimilarities += similarity_df[(similartags[y]):(similartags[y]+1)]

# usersimilarities / y
# usersimilarities
#usersimilarities + similarity_df[0:1].reset_index(drop=True)

In [12]:
usersimilarities = pd.DataFrame(0, index=[0], columns=similarity_df.columns) #initialize preferences to all 0
uinput = input('Enter a bill you like (0 to Exit): ')
counter = 1

while uinput != '0':
  # Find the index of the movie in the similarity dataframe
  try:
    movie_index = similarity_df.index.get_loc(uinput)
    counter += 1
    usersimilarities += similarity_df[movie_index:(movie_index+1)].reset_index(drop=True) #make sure indices align
  except KeyError:
    #want to look at tags instead
    print("Bill not in system, looking at tags")
    similartags = privatebilllist.loc[privatebilllist['Tags'].str.contains(uinput.lower(), case=False)].index.tolist()
    for x in privatebilllist.loc[privatebilllist['Title'].str.contains(uinput.lower(), case=False)].index.tolist():
      if x not in similartags:
        similartags.append(x) #add any keywords in title also

    for y in range(len(similartags)):
      usersimilarities += similarity_df[(similartags[y]):(similartags[y]+1)].reset_index(drop=True)
      counter += 1

  uinput = input('Enter a bill you like (0 to Exit): ')

#usersimilarities /= counter #running average

similarity_df = pd.concat([similarity_df, usersimilarities], ignore_index=False)

#sims = similarity_df.iloc[movie_index].sort_values(ascending=False)

sims = similarity_df.iloc[len(similarity_df)-1].sort_values(ascending=False)

#usersimilarities.sum().sum()

Bill not in system, looking at tags
Bill not in system, looking at tags
Bill not in system, looking at tags
Bill not in system, looking at tags
Bill not in system, looking at tags


In [13]:
top_10 = []
counter = 0 #counter for sims index
upto10 = 0 #counter to 10
while upto10 != 10:
  index = bills[bills['Title'] == sims.index[0:][counter]].index[0]
  if bills["Level"][index] != -1:
    top_10.append(sims.index[0:][counter])
    upto10 += 1

  counter += 1

# Print the top 5 most similar bills to the movie
print(f'Recommended Proposals:')
#print(top_10)
for x in range(len(top_10)):
  print(str(x+1) + '.\t' + top_10[x] + ', ' + bills['Date'][x])

Recommended Proposals:
1.	Online Safety and Cyberbullying Prevention Act, 2024-01-28
2.	Autonomous Vehicles Regulation and Safety Act, 2023-05-13
3.	Tax Relief for American Families and Workers Act of 2024, 2023-02-02
4.	Rural Broadband Expansion and Connectivity Program, 2021-06-30
5.	Universal Healthcare Access and Wellness Initiative, 2024-02-01
6.	Agricultural Sustainability and Innovation Program, 2023-12-20
7.	Space Exploration and Scientific Research Funding Act, 2022-10-02
8.	Digital Privacy and Data Protection Initiative, 2021-07-02
9.	Youth Empowerment and Leadership Development Act, 2023-05-03
10.	Ocean Conservation and Environmental Protection Act, 2022-08-15


In [14]:
similarity_df.drop(similarity_df.index[-1], inplace=True) #delete usersimilarities off the dataset
usersimilarities

,Tax Relief for American Families and Workers Act of 2024,Social and Economic Security Act of 2023,Health Crisis Response and Preparedness Act of 2023,National Obesity Prevention and Treatment Act of 2021,Online Safety and Cyberbullying Prevention Act,Energy Independence and Climate Resilience Act,California Cannabis Legalization and Regulation Act,Pennsylvania Tax Reform and Economic Growth Act,Sustainable University Practices and Environmental Initiative,National Infrastructure Modernization and Development Act,...,Clean Energy Innovation and Research Initiative,Community Policing and Police Accountability Act,Artificial Intelligence in Education and Integration Act,Worker's Rights Protection and Advocacy Initiative,Green Building and Sustainable Architecture Program,Mental Wellness and Suicide Prevention Program,Veterans Support and Healthcare Access Initiative,Smart Grid Infrastructure Modernization and Development Act,Humanitarian Aid and Refugee Support Program,Space Debris Cleanup and Environmental Protection Initiative
0,0.0,0.0,0.0,0.0,1.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
